<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [1]:
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

from sklearn.utils import shuffle

In [3]:
df = pd.read_csv('/datasets/Churn.csv')

In [4]:
df.head(5) #Выведим аблицу для ознокомления с ней.

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


Признаки

* RowNumber — индекс строки в данных
* CustomerId — уникальный идентификатор клиента
* Surname — фамилия
* CreditScore — кредитный рейтинг
* Geography — страна проживания
* Gender — пол
* Age — возраст
* Tenure — сколько лет человек является клиентом банка
* Balance — баланс на счёте
* NumOfProducts — количество продуктов банка, используемых клиентом
* HasCrCard — наличие кредитной карты
* IsActiveMember — активность клиента
* EstimatedSalary — предполагаемая зарплата

Целевой признак
* Exited — факт ухода клиента

In [5]:
df.info() # Посмотрим общую информацию.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [6]:
df.isna().sum() # Посмотрим пропуски.

RowNumber            0
CustomerId           0
Surname              0
CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64

Пропуски у нас только в строке Tenure (сколько лет человек является клиентом банка). Могу предположить, что это из за того, что меньше года присваивается nan. Давайте посмотрим так ли это.

In [7]:
df['Tenure'].value_counts() # Нет 0 есть тоже, значит причина в другом.

1.0     952
2.0     950
8.0     933
3.0     928
5.0     927
7.0     925
4.0     885
9.0     882
6.0     881
10.0    446
0.0     382
Name: Tenure, dtype: int64

In [8]:
print(df.duplicated().sum()) # Проверим есть ли явные дублекаты

0


Поскольку мне неизвестны причины отсутствия данных, а замена их на среднее или другие цифры могут повлиять на наши модели. С учетом, что пропусков около 9% было принято решение их удалить.

In [9]:
df = df.dropna(subset=['Tenure'])

In [10]:
df = df.reset_index(drop=True) #сброс индексов

Далее удалим столбци с данными которые непомогут нашему датасету в обучении.
* RowNumber — индекс строки в данных
* CustomerId — уникальный идентификатор клиента
* Surname — фамилия

In [11]:
df = df.drop(columns=['RowNumber', 'CustomerId', 'Surname'])

In [12]:
df.head(5) #Выведим таблицу для ознокомления с ней.

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


Подготовим библиотеку, данные и признаки — features и target. Разделим валидационную и обучающую выборки, в соотношении 60% на 40%. После 40% разделим пополам, чтобы получить еще тестовую выборку:

In [13]:
target = df['Exited']
features = df.drop('Exited', axis=1)
                       
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.4, random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(features_valid, target_valid, test_size=0.5, random_state=12345)

In [14]:
features_train = pd.get_dummies(features_train, drop_first=True)
features_valid = pd.get_dummies(features_valid, drop_first=True)
features_test = pd.get_dummies(features_test, drop_first=True)

Резюме.
* Удалили пропуски из строки Tenure
* Удалили столбци 'RowNumber', 'CustomerId', 'Surname'
* Разбили данные сначала на две части в 40% и 60%. После 40% еще пополам.

## Исследование задачи

In [15]:
df['Exited'].value_counts() # Соотношение оставшихся и ушедших клиентов.

0    7237
1    1854
Name: Exited, dtype: int64

Имеется дисбаланс классов. Примерно 1:4 в пользу оставшихся клиентов.

Обучение моделей:

Логистическая регрессия

In [16]:
model_reg =  LogisticRegression()
# Обучим модель вызовом метода fit().
model_reg.fit(features_train, target_train)
prediction = model_reg.predict(features_valid)
# Чтобы посмотреть accuracy,f1,AUC-ROC модели, нужно вызвать функцию .score():
print('Accuracy', accuracy_score(target_valid, prediction))

print('F1', f1_score(target_valid, prediction))
probabilities_valid = model_reg.predict_proba(features_valid)[:, 1]
print("AUC-ROC", roc_auc_score(target_valid, probabilities_valid))

Accuracy 0.7882288228822882
F1 0.06779661016949153
AUC-ROC 0.6902552567201159


Модель решающего дерева

In [17]:
best_tree_model = 0
best_tree_result = 0
for depth in range(3, 15):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth) # обучили модель с заданной глубиной дерева
    model_tree.fit(features_train, target_train) # обучили модель
    prediction = model_tree.predict(features_valid) # посчитали прогноз модели
    f1 = f1_score(target_valid, prediction)
    if f1>best_tree_model:
        best_tree_model=f1
        best_tree_result=depth
print('Max depth', best_tree_result,'F1-score', best_tree_model)

Max depth 7 F1-score 0.5764331210191083


In [18]:
model_tree = DecisionTreeClassifier(random_state=12345, max_depth=7)
model_tree.fit(features_train, target_train)
prediction = model_tree.predict(features_valid)
print('Accuracy', accuracy_score(target_valid, prediction))


print('F1', f1_score(target_valid, prediction))
probabilities_valid = model_tree.predict_proba(features_valid)[:, 1]
print("AUC-ROC", roc_auc_score(target_valid, probabilities_valid))

Accuracy 0.8536853685368537
F1 0.5764331210191083
AUC-ROC 0.8346049843812412


Случайный лес

In [19]:
f1_best = 0
estim_best = 0
depth_best = 0

for estim in range (20, 30):
    for depth in range(11, 15):
        model = RandomForestClassifier(n_estimators=estim, max_depth=depth, random_state=12345)
        model.fit(features_train,target_train)
        prediction = model.predict(features_valid)
        f1 = f1_score(target_valid, prediction)
        if f1>f1_best:
            f1_best=f1
            estim_best=estim
            depth_best=depth
        

print('n_estimators =', estim_best, 'Max depth', depth_best, 'F1-score', f1_best)

n_estimators = 24 Max depth 14 F1-score 0.5886287625418061


In [20]:
model = RandomForestClassifier(n_estimators=24, max_depth=14, random_state=12345)
model.fit(features_train,target_train)
prediction = model.predict(features_valid)
print('Accuracy', accuracy_score(target_valid, prediction))

print('F1', f1_score(target_valid, prediction))
probabilities_valid = model.predict_proba(features_valid)[:, 1]
print("AUC-ROC", roc_auc_score(target_valid, probabilities_valid))

Accuracy 0.8646864686468647
F1 0.5886287625418061
AUC-ROC 0.8498206189703952


Резюме.

Самый лучший результат из трех моделий показал случайный лес F1 0.588. Почьти нужное значение

## Борьба с дисбалансом

Поборемся с дисбалансом классов с помощью атрибута class_weight.

In [21]:
# Логистическая регрессия
model = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.5018587360594795


In [22]:
# решающее дерева
model_balanced = DecisionTreeClassifier(random_state=12345, max_depth=5, class_weight = 'balanced')
model_balanced.fit(features_train, target_train)

predicted_valid = model_balanced.predict(features_valid)
f1_balanced = f1_score(target_valid, predicted_valid)
print("F1:", f1_balanced)

F1: 0.5735449735449736


In [23]:
# Случайный лес
model_balanced1 = RandomForestClassifier(n_estimators=8, random_state=12345, max_depth=11, class_weight = 'balanced')
model_balanced1.fit(features_train, target_train)

predicted_valid1 = model_balanced1.predict(features_valid)
f1_balanced1 = f1_score(target_valid, predicted_valid1)
print("F1:", f1_balanced1)

F1: 0.6205997392438071


Лучший результат показал случайный лес F1: 0.62 В чек-листе готовности проекта есть "Применено несколько способов борьбы с дисбалансом". Примененим несколько способов борьбы с дисбалансом.

Увеличение выборки. Когда обучают модели, такая техника называется upsampling (от англ. up, «вверх»; sampling, «выборка»).

In [24]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 10)



In [25]:
# Логистическая регрессия
model_ups_reg = LogisticRegression(random_state=12345, solver='liblinear')
model_ups_reg.fit(features_upsampled, target_upsampled)
predicted_valid_reg = model_ups_reg.predict(features_valid)
print("F1:", f1_score(target_valid, predicted_valid_reg))

F1: 0.3639067922657412


Результат стал хуже, чем в работе с помощью атрибута class_weight.

In [26]:
# решающее дерева
model_ups_tree = DecisionTreeClassifier(random_state=12345, max_depth=6)
model_ups_tree.fit(features_upsampled, target_upsampled)
predicted_valid_tree = model_ups_tree.predict(features_valid)
f1_ups_tree = f1_score(target_valid, predicted_valid_tree)
print("F1:", f1_ups_tree)

F1: 0.5361344537815127


Результат стал хуже, чем в работе с помощью атрибута class_weight.

In [27]:
# Случайный лес
model_ups_forest = RandomForestClassifier(n_estimators=10, random_state=12345, max_depth=15)
model_ups_forest.fit(features_upsampled, target_upsampled)

predicted_valid_forest = model_ups_forest.predict(features_valid)
f1_ups_forest = f1_score(target_valid, predicted_valid_forest)
print("F1:", f1_ups_forest)

F1: 0.5901201602136181


Результат стал хуже, чем в работе с помощью атрибута class_weight.

Резюме.

Самый лучший результат показал случайный лес F1: 0.62 к которому применили атрибут class_weight.

## Тестирование модели

Проведем обучение после балансировки данных

Теперь обучим модель случайного леса поскольку она показала наилучший результат.

In [28]:
model = RandomForestClassifier(n_estimators=44, random_state=12345, max_depth=14,  class_weight = 'balanced')
model.fit(features_train, target_train)

predicted_test = model.predict(features_test)
f1 = f1_score(target_test, predicted_test)
print("F1:", f1)

F1: 0.6105610561056105


In [29]:
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]


auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print(auc_roc)

0.8610234198546912


Значение метрики F1 равен 0.61, что выше заявленного минимума 0.59 в задании, так же значение auc_roc будет 0.86, что говорит о неслучайности наших значений.

# Вывод

Выполняя работу мы спрогнозировали, уйдёт клиент из банка в ближайшее время или нет. С помощью машинного обучения построили модель со значением F1-меры, свыше 0.59, что дает достаточный уровень F1. Для этого была проведена следующая работа. В первичном датасете присутствовали пропуски их решили удалить, так как отсутствие 9% данным не привело бы к значимому ухудшению качества модели, а заполнение этих пропусков средним значением или медианой возможно привело бы к ложным предсказанием на практики. Далее были удалены столбцы, не участвующие в работе модели ('RowNumber', 'CustomerId', 'Surname'). Весь исходный датасет был разделен на обучающую - 60%, валтдационную - 20% и тестовую - 20% выборки. В данных имеется дисбаланс классов, примерно 1:4 в пользу оставшихся клиентов. Проверка проходила на трех моделях (Логистическая регрессия, дерева решений, случайный лес). В начале создаем модель и обучаем ее на обучающей выборке, далее настраиваем гиперпараметры и проверяем точность на валидационной. Также мы боролись с дисбалансом классов с помощью атрибута class_weight, что значительно увеличило наши покзатели и использовали другой метод, увеличение выборки (upsampling) который показал себя хуже чем class_weight. Лучшие показатели модели были у случайного леса поэтому ее использовали для тестовой выборки. Результаты проверки модели на тестовой выборке показали, что модель обладает значением F1 0.61, что выше заявленных пороговых значениий, а значит соответствует заданию. Метрика auc_roc имеет значение 0.86, что выше по сравнению с F1-мерой.